Задание 1

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса 

https://www.cbr-xml-daily.ru/daily_json.js

In [1]:
import requests

In [43]:
class Rate:
    """
    
    """
    def __init__(self, format_='value'):
        self.format = format_
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [5]:
class NameCurrency(Rate):
    def __init__(self):
        super().__init__(format_='full')
    
    def name_of_currency(self, currency):
        return self.make_format(currency)['Name']    

In [44]:
r = NameCurrency()
r.name_of_currency('USD')

'Доллар США'

Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. 
Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [45]:
class Rate:
    def __init__(self, format_='value', diff = 'False'):
        self.format = format_
        self.diff = diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает изменение по сравнению в прошлым значением"""
        if self.diff == 'True' and self.format != 'full':
            data_of_valute = self.exchange_rates()
            previous = data_of_valute['EUR']['Previous']
            change_valute = round((self.make_format('EUR') - previous), 4)
            return change_valute
        else:
            return self.make_format('EUR')
    
    def usd(self):
        """Возвращает изменение по сравнению в прошлым значением"""
        if self.diff == 'True' and self.format != 'full':
            data_of_valute = self.exchange_rates()
            previous = data_of_valute['USD']['Previous']
            change_valute = round((self.make_format('USD') - previous), 4)
            return change_valute
        else:
            return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [46]:
r_diff = Rate(diff = 'True')
r_diff.eur()

-0.1202

In [47]:
r_full = Rate(format_ = 'full', diff = 'True')
r_full.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 61.5333,
 'Previous': 61.5694}

# Домашнее задание задача 3

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

Для проверки используйте код:

In [106]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [107]:
class Designer(Employee):
    def __init__(self, name, seniority, award = 0):
        super().__init__(name, seniority)
        self.award = award
    
    def check_if_it_is_time_for_upgrade(self):
        self.seniority += 1
        seniority_ = self.award + self.seniority
        # условие повышения сотрудника из презентации
        if seniority_ % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [108]:
elena = Designer(name = 'Елена', seniority=0, award = 2)

In [109]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена 1
Елена 1
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 4
Елена 4
